<a href="https://colab.research.google.com/github/araga2004/Image_Caption_Gen/blob/main/testing_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from pickle import load
from PIL import Image, UnidentifiedImageError
import numpy as np
import os

# Get the image path from user input
full_img_path = input("Input path to your image (relative to My Drive): ")

if not os.path.isfile(full_img_path):
    print(f"ERROR: File not found! Make sure the image path '{full_img_path}' is correct")
else:
    print(f"File found at: {full_img_path}")

    def extract_features(image_path, model):
        try:
            img = Image.open(image_path)
        except FileNotFoundError:
            print(f"ERROR: File not found! Make sure the image path '{image_path}' is correct")
            return None
        except UnidentifiedImageError:
            print(f"ERROR: Cannot identify image file! Make sure the image at '{image_path}' is valid")
            return None
        except Exception as e:
            print(f"ERROR: An unexpected error occurred while opening the image: {e}")
            return None

        if img.mode == 'RGBA':
            img = img.convert('RGB')
        img = img.resize((299, 299))
        img = np.array(img)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)
        feature = model.predict(img)
        return feature

    def generate_desc(model, tokenizer, photo, max_length):
        in_text = 'start'

        for _ in range(max_length):
            sequence = tokenizer.texts_to_sequences([in_text])[0]
            sequence = pad_sequences([sequence], maxlen=max_length)
            pred = model.predict([photo, sequence], verbose=0)
            pred = np.argmax(pred)

            word = word_for_id(pred, tokenizer)
            if word is None or word == 'end':
                break
            in_text += ' ' + word

        return in_text

    def word_for_id(integer, tokenizer):
        for word, index in tokenizer.word_index.items():
            if index == integer:
                return word
        return None

    max_length = 32

    # Load the tokenizer
    with open('/content/drive/My Drive/tokenizer1.p', 'rb') as f:
        tokenizer = load(f)

    model = load_model('/content/drive/My Drive/model_9.h5')
    xception_model = Xception(include_top=False, pooling="avg")

    photo = extract_features(full_img_path, xception_model)
    if photo is not None:
        description = generate_desc(model, tokenizer, photo, max_length)
        print("Caption: ")
        print(description)
    else:
        print("Feature extraction failed.")


Input path to your image (relative to My Drive): /content/drive/My Drive/IMG-20240630-WA0002.jpg
File found at: /content/drive/My Drive/IMG-20240630-WA0002.jpg
1/1 [==============================] - 2s 2s/step
Caption: 
start man in red shirt is walking down the street


In [20]:
# Get the image path from user input
import os
img_path = input("Input path to your image (relative to My Drive): ")
full_img_path = os.path.join('/content/drive/My Drive', img_path)
print(full_img_path)
# Check if the file exists
if not os.path.isfile(full_img_path):
    print(f"ERROR: File not found! Make sure the image path '{full_img_path}' is correct")
else:
    print(f"File found at: {full_img_path}")

# Proceed with the rest of the code if the file is found


Input path to your image (relative to My Drive): /99171998_7cc800ceef.jpg
/99171998_7cc800ceef.jpg
ERROR: File not found! Make sure the image path '/99171998_7cc800ceef.jpg' is correct
